In [1]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar
from pypdf import PdfReader
import re

# ***Tạo hàm xử lý các chun***

In [ ]:
# Xác định format của chunk được sử dụng:

# {
#   "id": "3-2-1",
#   "values": [0.123, -0.456, ...],
#   "metadata": {
#     "subject": "Lịch sử ĐCSVN",
#     "chapter": "Chương III",
#     "chapter_title": "Chương III: Đường lối kháng chiến",
#     "section": "II.",
#     "section_title": "II. Giai đoạn đầu",
#     "subsection": "1.",
#     "subsection_title": "1. Tư tưởng chỉ đạo",
#     "content": "...",
#     "position_in_document": 73,
#     "tokens": 296,
#     "type": "theory"
#   }
# }

In [31]:
def parse_chapters(text):
    """
    Hàm được sử dụng để tách 1 đoạn chương thành 2 thành phần là số chương và title của chương đó
    """
    chapters_data = []
    pattern = re.compile(r"(Chương\s(?:nhập môn|\d+))\s*(.*)")

    for line in text.strip().split('\n'):
        match = pattern.match(line)
        if match:
            chapter_number = match.group(1).strip()
            chapter_title = match.group(2).strip()
            chapters_data.append({
                'chapter': chapter_number,
                'chapter_title': chapter_title
            })
    return chapter_number, chapter_title

In [32]:
def build_chunk(
    chapter_line,
    section_line,
    sub_section_line,
    content_line,
    position_in_document=0,
    model=None  # model dùng để embedding nếu muốn
):
    if chapter_line != "":
        chapter_number, chapter_title = parse_chapters(chapter_line)
    else:
        chapter_number, chapter_title = None

    # Xử lý mục (section)
    if section_line != "":
        section = section_line.strip().split()[0]  # lấy "II."
        section_title = section_line.strip()
    else:
        section = None
        section_title = None

    # Xử lý tiểu mục (subsection)
    if sub_section_line != "":
        subsection = sub_section_line.strip().split()[0]  # lấy "1."
        subsection_title = sub_section_line.strip()
    else:
        subsection = None
        subsection_title = None
        
    return chapter_number, chapter_title, section, section_title, subsection, subsection_title

# ***Xử lý các phần chú thích ở cuối trang***

In [2]:
pdf_path = "../data/LichSuDang/lich_su_dang.pdf"

#Tạo fulltext để chứa toàn bộ các dòng chử trong file Lịch sử Đảng
sections = []

# Lấy số trang trong file PDF
reader = PdfReader(pdf_path)
total_pages = len(reader.pages)
print("Số trang trong PDF:", total_pages)

Số trang trong PDF: 219


In [3]:
pdf_path = "../data/LichSuDang/lich_su_dang.pdf"

def chunking_LSD(pdf_path):
    chapter_title = []
    section_title = []
    sub_section_title = []
    content = []

    is_content = False

    curr_chapter_line = ""
    curr_section_line = ""
    curr_sub_section_line = ""
    curr_content_line = ""

    #Duyệt qua từng trang trong file pdf, bắt đầu từ trang 2 (bỏ qua trang bìa)
    for page_layout in extract_pages(pdf_path, page_numbers=range(2, total_pages)):
        
        #Duyệt qua từng phần tử trong trang
        for element in page_layout:
            if isinstance(element, LTTextContainer):

                # Duyet qua từng dòng chữ trong phần tử
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        
                        #Lấy ra từng kí tự bên trong dòng chữ
                        for obj in text_line:
                            if isinstance(obj, LTChar):
                                '''
                                Dựa vào kích thước chữ để phân loại các phần trong tài liệu
                                - Kích thước chữ = 14: Chương và tiêu đề của chương - Done
                                - Kích thước chữ = 13, vừa in đậm vừa in nghiêng: Tiêu đề phụ (1 nhỏ, 2 nhỏ, 3 nhỏ, ...) - Done
                                - Kích thước chữ = 13, chỉ in đậm không in nghiêng: Tiêu đề lớn ( 1 la mã, 2 la mã, 3 la mã, ...), Mục Tiêu ( Chú ý xử lý trường hợp này) - Done
                                - Kích thước chữ > 12.9: Các nội dung chính
                                - Kích thước chữ = 12: Số trang - Done
                                - Kích thước chữ < 11: Nội dung phụ, chú thích, ghi chú (Loại bỏ các trường hợp này) - Done
                                '''
                                #Lấy ra thông tin của kí tự
                                font = obj.fontname
                                text = obj.get_text()
                                text_size = obj.size

                                if text_size == 14:    #Lấy ra các ký tự có kích thước chữ = 14, đây là tiêu đề chương

                                    #Nếu có nội dung chương trước đó thì đẩy 1 chunk
                                    if is_content == True:
                                        print(curr_chapter_line)
                                        print("1")
                                        print(curr_chapter_line)
                                        print(curr_section_line)
                                        print(curr_sub_section_line)
                                        is_content = False

                                    # curr_chapter_line =""
                                    curr_chapter_line += text

                                elif text_size >12.9 and text_size < 14:   #Lấy ra các ký tự có kích thước chữ > 12.9 (Đây là nội dung chính)

                                    #Tiêu đề la mã
                                    if ("Bold" in font) and ("Italic" not in font):
                                        #Nếu có nội dung chương trước đó thì đẩy 1 chunk
                                        if is_content == True:
                                            print(curr_chapter_line)
                                            print(curr_section_line)
                                            print(curr_sub_section_line)
                                            is_content = False

                                        # curr_section_line = ""
                                        curr_section_line += text
                                    
                                    #Tiêu đề phụ
                                    elif ("Italic" in font) and ("Bold" in font):
                                        #Nếu có nội dung chương trước đó thì đẩy 1 chunk
                                        if is_content == True:
                                            print(curr_chapter_line)
                                            print(curr_section_line)
                                            print(curr_sub_section_line)
                                            is_content = False

                                        # curr_sub_section_line =""
                                        curr_sub_section_line += text
                                    
                                    #Trường hợp là văn bản bình thường
                                    else:
                                        is_content = True
                                        curr_content_line += text
                        # print("Chapter:", curr_chapter_line)                    
                        # print("Section", curr_section_line)                    
                        # print("Subsection:", curr_sub_section_line) 
                        # print("==================================")                   
            # if curr_section_line.strip():  # chỉ thêm nếu có ký tự hợp lệ
            #     section_title.append(curr_section_line)
            #     curr_section_line = ""
            # if curr_sub_section_line.strip():  # chỉ thêm nếu có ký tự hợp lệ
            #     sub_section_title.append(curr_sub_section_line)
            #     curr_sub_section_line = ""
            # if curr_content_line.strip():  # chỉ thêm nếu có ký tự hợp lệ
            #     content.append(curr_content_line)
            #     curr_content_line = ""
    return chapter_title, section_title, sub_section_title, content

In [4]:
chapter_title, section_title, sub_section_title, content = chunking_LSD(pdf_path)

Chương nhập mônĐỐI TƯỢNG, CHỨC NĂNG, NHIỆM VỤ,NỘI DUNG VÀ PHƯƠNG PHÁP NGHIÊN CỨU, HỌC TẬPLỊCH SỬ ĐẢNG CỘNG SẢN VIỆT NAM


Chương nhập mônĐỐI TƯỢNG, CHỨC NĂNG, NHIỆM VỤ,NỘI DUNG VÀ PHƯƠNG PHÁP NGHIÊN CỨU, HỌC TẬPLỊCH SỬ ĐẢNG CỘNG SẢN VIỆT NAM
I. Đối tượng nghiên cứu của môn học Lịch sử Đảng Cộng sản Việt Nam 

Chương nhập mônĐỐI TƯỢNG, CHỨC NĂNG, NHIỆM VỤ,NỘI DUNG VÀ PHƯƠNG PHÁP NGHIÊN CỨU, HỌC TẬPLỊCH SỬ ĐẢNG CỘNG SẢN VIỆT NAM
I. Đối tượng nghiên cứu của môn học Lịch sử Đảng Cộng sản Việt Nam 1.

Chương nhập mônĐỐI TƯỢNG, CHỨC NĂNG, NHIỆM VỤ,NỘI DUNG VÀ PHƯƠNG PHÁP NGHIÊN CỨU, HỌC TẬPLỊCH SỬ ĐẢNG CỘNG SẢN VIỆT NAM
I. Đối tượng nghiên cứu của môn học Lịch sử Đảng Cộng sản Việt Nam 1.2.

Chương nhập mônĐỐI TƯỢNG, CHỨC NĂNG, NHIỆM VỤ,NỘI DUNG VÀ PHƯƠNG PHÁP NGHIÊN CỨU, HỌC TẬPLỊCH SỬ ĐẢNG CỘNG SẢN VIỆT NAM
I. Đối tượng nghiên cứu của môn học Lịch sử Đảng Cộng sản Việt Nam 1.2.3.

Chương nhập mônĐỐI TƯỢNG, CHỨC NĂNG, NHIỆM VỤ,NỘI DUNG VÀ PHƯƠNG PHÁP NGHIÊN CỨU, HỌC TẬPLỊCH SỬ ĐẢNG CỘNG SẢN

In [5]:
chapter_title

[]

In [6]:
section_title

[]

In [7]:
sub_section_title

[]

In [8]:
content

[]

In [37]:
chapter_title = []
section_title = []
sub_section_title = []
content = []

is_content = False

curr_chapter_line = ""
curr_section_line = ""
curr_sub_section_line = ""
curr_content_line = ""

#Duyệt qua từng trang trong file pdf, bắt đầu từ trang 2 (bỏ qua trang bìa)
for page_layout in extract_pages(pdf_path, page_numbers=range(2, total_pages)):
    
    #Duyệt qua từng phần tử trong trang
    for element in page_layout:
        if isinstance(element, LTTextContainer):

            # Duyet qua từng dòng chữ trong phần tử
            for text_line in element:
                if isinstance(text_line, LTTextLine):
                    
                    #Lấy ra từng kí tự bên trong dòng chữ
                    for obj in text_line:
                        if isinstance(obj, LTChar):
                            '''
                            Dựa vào kích thước chữ để phân loại các phần trong tài liệu
                            - Kích thước chữ = 14: Chương và tiêu đề của chương - Done
                            - Kích thước chữ = 13, vừa in đậm vừa in nghiêng: Tiêu đề phụ (1 nhỏ, 2 nhỏ, 3 nhỏ, ...) - Done
                            - Kích thước chữ = 13, chỉ in đậm không in nghiêng: Tiêu đề lớn ( 1 la mã, 2 la mã, 3 la mã, ...), Mục Tiêu ( Chú ý xử lý trường hợp này) - Done
                            - Kích thước chữ > 12.9: Các nội dung chính
                            - Kích thước chữ = 12: Số trang - Done
                            - Kích thước chữ < 11: Nội dung phụ, chú thích, ghi chú (Loại bỏ các trường hợp này) - Done
                            '''
                            #Lấy ra thông tin của kí tự
                            font = obj.fontname
                            text = obj.get_text()
                            text_size = obj.size

                            if text_size == 14:    #Lấy ra các ký tự có kích thước chữ = 14, đây là tiêu đề chương

                                #Nếu có nội dung chương trước đó thì đẩy 1 chunk
                                if is_content == True:
                                    chapter_number, chapter_title, section, section_title, subsection, subsection_title = build_chunk(curr_chapter_line, curr_section_line, curr_sub_section_line, content)
                                    print("Chương hiện tại:       ", chapter_number)
                                    print("Tiêu đề chương:        ", chapter_title)
                                    print("Mục hiện tại:          ", section)
                                    print("Tiêu đề mục:           ", section_title)
                                    print("Mục con hiện tại:      ", subsection)
                                    print("Tiêu đề mục con:       ", subsection_title)
                                    print("Nội dung đoạn:         ", curr_content_line[:20], "...", curr_content_line[-20:], "------------", len(curr_content_line))
                                    print("======================================================================================================") 
                                    curr_chapter_line = ""
                                    curr_section_line = ""
                                    curr_sub_section_line = ""
                                    curr_content_line = ""
                                    is_content = False

                                # curr_chapter_line =""
                                curr_chapter_line += text

                            elif text_size >12.9 and text_size < 14:   #Lấy ra các ký tự có kích thước chữ > 12.9 (Đây là nội dung chính)

                                #Tiêu đề la mã
                                if ("Bold" in font) and ("Italic" not in font):
                                    #Nếu có nội dung chương trước đó thì đẩy 1 chunk
                                    if is_content == True:
                                        chapter_number, chapter_title, section, section_title, subsection, subsection_title = build_chunk(curr_chapter_line, curr_section_line, curr_sub_section_line, content)
                                        print("Chap hien tai:", chapter)
                                        print("Chương hiện tại:       ", chapter_number)
                                        print("Tiêu đề chương:        ", chapter_title)
                                        print("Mục hiện tại:          ", section)
                                        print("Tiêu đề mục:           ", section_title)
                                        print("Mục con hiện tại:      ", subsection)
                                        print("Tiêu đề mục con:       ", subsection_title)
                                        print("Nội dung đoạn:         ", curr_content_line[:20], "...", curr_content_line[-20:], "------------", len(curr_content_line))
                                        print("======================================================================================================") 
                                        curr_section_line = ""
                                        curr_sub_section_line = ""
                                        curr_content_line = ""
                                        is_content = False

                                    # curr_section_line = ""
                                    curr_section_line += text
                                
                                #Tiêu đề phụ
                                elif ("Italic" in font) and ("Bold" in font):
                                    #Nếu có nội dung chương trước đó thì đẩy 1 chunk
                                    if is_content == True:
                                        chapter_number, chapter_title, section, section_title, subsection, subsection_title = build_chunk(curr_chapter_line, curr_section_line, curr_sub_section_line, content)
                                        print("Chương hiện tại:       ", chapter_number)
                                        print("Tiêu đề chương:        ", chapter_title)
                                        print("Mục hiện tại:          ", section)
                                        print("Tiêu đề mục:           ", section_title)
                                        print("Mục con hiện tại:      ", subsection)
                                        print("Tiêu đề mục con:       ", subsection_title)
                                        print("Nội dung đoạn:         ", curr_content_line[:20], "...", curr_content_line[-20:], "------------", len(curr_content_line))
                                        print("======================================================================================================") 
                                        curr_sub_section_line = ""
                                        curr_content_line = ""
                                        is_content = False

                                    # curr_sub_section_line =""
                                    curr_sub_section_line += text
                                
                                #Trường hợp là văn bản bình thường
                                else:
                                    is_content = True
                                    curr_content_line += text
                    # print("Chapter:", curr_chapter_line)                    
                    # print("Section", curr_section_line)                    
                    # print("Subsection:", curr_sub_section_line) 
                    # print("======================================================================================================")                   
        # if curr_section_line.strip():  # chỉ thêm nếu có ký tự hợp lệ
        #     section_title.append(curr_section_line)
        #     curr_section_line = ""
        # if curr_sub_section_line.strip():  # chỉ thêm nếu có ký tự hợp lệ
        #     sub_section_title.append(curr_sub_section_line)
        #     curr_sub_section_line = ""
        # if curr_content_line.strip():  # chỉ thêm nếu có ký tự hợp lệ
        #     content.append(curr_content_line)
        #     curr_content_line = ""
# return chapter_title, section_title, sub_section_title, content

Chap hien tai: Chương 3ĐẢNG LÃNH ĐẠO CẢ NƯỚC QUÁ ĐỘ LÊNCHỦ NGHĨA XÃ HỘI VÀ TIẾN HÀNH CÔNG CUỘC ĐỔI MỚI(1975 - 2018)
Chương hiện tại:        Chương nhập môn
Tiêu đề chương:         ĐỐI TƯỢNG, CHỨC NĂNG, NHIỆM VỤ,NỘI DUNG VÀ PHƯƠNG PHÁP NGHIÊN CỨU, HỌC TẬPLỊCH SỬ ĐẢNG CỘNG SẢN VIỆT NAM
Mục hiện tại:           None
Tiêu đề mục:            None
Mục con hiện tại:       None
Tiêu đề mục con:        None
Nội dung đoạn:          Đảng Cộng sản Việt N ... o quan điểm củaĐảng. ------------ 2168
Chap hien tai: Chương 3ĐẢNG LÃNH ĐẠO CẢ NƯỚC QUÁ ĐỘ LÊNCHỦ NGHĨA XÃ HỘI VÀ TIẾN HÀNH CÔNG CUỘC ĐỔI MỚI(1975 - 2018)
Chương hiện tại:        Chương nhập môn
Tiêu đề chương:         ĐỐI TƯỢNG, CHỨC NĂNG, NHIỆM VỤ,NỘI DUNG VÀ PHƯƠNG PHÁP NGHIÊN CỨU, HỌC TẬPLỊCH SỬ ĐẢNG CỘNG SẢN VIỆT NAM
Mục hiện tại:           I.
Tiêu đề mục:            I. Đối tượng nghiên cứu của môn học Lịch sử Đảng Cộng sản Việt Nam
Mục con hiện tại:       None
Tiêu đề mục con:        None
Nội dung đoạn:          Đối tượng nghiên cứu ... c

IndexError: list index out of range

Các fontsize có trong giáo trình:
    - {5.799999999999997,
        7.5,
        7.500000000000028,
        9.999999999999986,
        10.0,
        10.000000000000007,
        11.0,
        11.000000000000007,
        12.0,
        12.999999999999943,
        12.999999999999972,
        12.999999999999993,
        13.0,
        13.000000000000007,
        14.0,
        14.000000000000007,
        14.000000000000014,
        16.0}